## Анализ музыкальных предпочтений пользователей Яндекс.Музыка

Датасет содержит информацию о пользователях стримингового сервиса "Яндекс. Музыка" в Москве и Санкт-Петербурге.
<p>Необходимо осуществить предобработку данных и проверить гипотезы:
    <br>1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
    <br>2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
    <br>3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.
</p>

Значения столбцов датасета:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

## 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/yandex_music_project.csv')
df.head(5)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Исползуется *camelCase*, вместо *snake_case*.


Количество значений в столбцах различается, то есть в данных есть пропущенные значения.

## 2. Предобработка данных

### 2.1. Форматирование заголовков

In [4]:
# Перечень названий столбцов таблицы df.

df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [5]:
# Поскольку название столбцов используют разный стиль форматирование, переименуем их.

df = df.rename(columns={'  userID': 'user_id',
                        'Track': 'track',
                        '  City  ': 'city',
                        'Day': 'day'
                       })

### 2.2.Обработка пропущенных значений

In [6]:
# Подсчёт пропусков.

df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [7]:
# Перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'.

columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')


    
# Проверка: подсчёт пропусков.

df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### 2.3. Обработка дубликатов

In [8]:
# Число явных дубликатов.

df.duplicated().sum()

3826

In [9]:
# Удаление явных дубликатов (с удалением старых индексов и формированием новых).

df = df.drop_duplicates().reset_index(drop=True)

In [10]:
# Проверка: число явных дубликатов.

df.duplicated().sum()

0

Избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано по-разному.

In [11]:
# Просмотр уникальных названий жанров.

df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Жанр hiphop записан по-разному ('hip', 'hop', 'hip-hop'). Исправим это.

In [12]:
# Функция для замены неявных дубликатов.

def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

genre_incorrect = ['hip', 'hop', 'hip-hop']
genre_correction = 'hiphop'

In [13]:
# Устранение неявных дубликатов.

replace_wrong_genres(genre_incorrect, genre_correction)

In [14]:
# Проверка: неявные дубликаты.

df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных, которые были исправлены:
- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

## 3. Проверка гипотез

### 3.1. Сравнение поведения пользователей двух городов

Предположим, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим эту гипотезу на данных о трёх днях недели — понедельнике, среде и пятнице.
<p>Для этого:
    
* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.
</p>

In [15]:
# Подсчёт прослушиваний в каждом городе.

print(df.groupby('city')['track'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [16]:
# Подсчёт прослушиваний в каждый из трёх дней.

df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [17]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    
    return track_list_count

In [42]:
print('Количество прослушиваний в Москве по понедельникам:', number_tracks('Monday', 'Moscow'))
print('Количество прослушиваний в Санкт-Петербурге по понедельникам', number_tracks('Monday', 'Saint-Petersburg'), '\n')
print('Количество прослушиваний в Москве по средам', number_tracks('Wednesday', 'Moscow'))
print('Количество прослушиваний в Санкт-Петербурге по средам', number_tracks('Wednesday', 'Saint-Petersburg'), '\n')
print('Количество прослушиваний в Москве по пятницам', number_tracks('Friday', 'Moscow'))
print('Количество прослушиваний в Санкт-Петербурге по пятницам', number_tracks('Friday', 'Saint-Petersburg'))

Количество прослушиваний в Москве по понедельникам: 15740
Количество прослушиваний в Санкт-Петербурге по понедельникам 5614 

Количество прослушиваний в Москве по средам 11056
Количество прослушиваний в Санкт-Петербурге по средам 7003 

Количество прослушиваний в Москве по пятницам 15945
Количество прослушиваний в Санкт-Петербурге по пятницам 5895


In [44]:
# Приведем эти данные к табличной форме.

columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow', 15740, 11056, 15945],
       ['Saint-Petersburg', 5614, 7003, 5895]]

result = pd.DataFrame(data=data, columns=columns)
display(result)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 3.1 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [25]:
# получение таблиц по городам.

moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

In [27]:
# Функция, возвращающая информацию о топ-10 жанров тех треков, которые прослушивали в указанный день,
# в промежутке между двумя отметками времени.

def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).

In [47]:
print(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'), '\n\n')
print(genre_weekday(spb_general, 'Monday', '07:00', '11:00'), '\n\n')
print(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'), '\n\n')
print(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64 


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64 


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64 


genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно увидеть следующее:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.1 Жанровые предпочтения в Москве и Петербурге

Гипотеза: в Петербурге hiphop слушают чаще, чем в Москве; в Москве же преобладает поп-музыка.

In [50]:
# Сгруппируем жанры, прослушиваемые в Москве.

moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [51]:
# Сгруппируем жанры, прослушиваемые в Петербурге.

spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Рэп одинаково популярен в Москве и Петербурге. 


## 4. Итоги исследования

В результате исследования, были сформулирвоаны следующие выводы:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.